In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [7]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [10]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

330


In [11]:
df = pd.read_csv("../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형태소만).csv", encoding='UTF-8')
df

,split_str,org_idx,correct_str,morphs
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.,"['블로그', 'mom', '5', '개', '의', '글', 'mom', '목록'..."
1,momo 2018 12 31 19 52 https blog naver com omj...,0,"mom 2018, 12, 31, 19, 52 https blog NAVER com ...","['mom', '2018', ',', '12', ',', '31', ',', '19..."
2,An enemy generally says and believes what he w...,0,An enemy generally says and believes what he w...,"['An', 'enemy', 'generally', 'says', 'and', 'b..."
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0,True love is the joy of life 진실한 사랑은 인생의 환희다,"['True', 'love', 'is', 'the', 'joy', 'of', 'li..."
4,Carpe diem 현재를 즐겨라,0,Crape idem 현재를 즐겨라.,"['Crape', 'idem', '현재', '를', '즐겨라', '.']"
...,...,...,...,...
135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다,5170,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.,"['한편', '경계', '의', '의미', '로', '이', '사자성어', '를',..."
135820,조은영 원광대 교수 미술과 는 2017년을 종합하기에는 수락석출 외의 단어들이 지나...,5170,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...,"['조은영', '원광대', '교수', '미술', '과는', '2017년', '을',..."
135821,올해의 사자성어는 3위부터 5위까지는 약 16 대의 고른 분포를 보인 것이 특징이다,5170,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.,"['올해', '의', '사자성어', '는', '3', '위', '부터', '5', ..."
135822,4위는 16 5위는 15 1 였다,5170,"4위는 16, 5위는 15, 1 쳤다","['4', '위', '는', '16', ',', '5', '위', '는', '15'..."


In [12]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,correct_str,morphs,joined_morphs
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.,"['블로그', 'mom', '5', '개', '의', '글', 'mom', '목록'...",블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 .
1,momo 2018 12 31 19 52 https blog naver com omj...,0,"mom 2018, 12, 31, 19, 52 https blog NAVER com ...","['mom', '2018', ',', '12', ',', '31', ',', '19...","mom 2018 , 12 , 31 , 19 , 52 https blog NAVER ..."
2,An enemy generally says and believes what he w...,0,An enemy generally says and believes what he w...,"['An', 'enemy', 'generally', 'says', 'and', 'b...",An enemy generally says and believes what he w...
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0,True love is the joy of life 진실한 사랑은 인생의 환희다,"['True', 'love', 'is', 'the', 'joy', 'of', 'li...",True love is the joy of life 진실한 사랑 은 인생 의 환희 다
4,Carpe diem 현재를 즐겨라,0,Crape idem 현재를 즐겨라.,"['Crape', 'idem', '현재', '를', '즐겨라', '.']",Crape idem 현재 를 즐겨라 .
...,...,...,...,...,...
135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다,5170,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.,"['한편', '경계', '의', '의미', '로', '이', '사자성어', '를',...",한편 경계 의 의미 로 이 사자성어 를 추천 한 이 들 도 눈 에 띈다 .
135820,조은영 원광대 교수 미술과 는 2017년을 종합하기에는 수락석출 외의 단어들이 지나...,5170,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...,"['조은영', '원광대', '교수', '미술', '과는', '2017년', '을',...",조은영 원광대 교수 미술 과는 2017년 을 종합 하기에는 수락 석출 외 의 단어 ...
135821,올해의 사자성어는 3위부터 5위까지는 약 16 대의 고른 분포를 보인 것이 특징이다,5170,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.,"['올해', '의', '사자성어', '는', '3', '위', '부터', '5', ...",올해 의 사자성어 는 3 위 부터 5 위 까지는 약 16 대의 고른 분포 를 보인 ...
135822,4위는 16 5위는 15 1 였다,5170,"4위는 16, 5위는 15, 1 쳤다","['4', '위', '는', '16', ',', '5', '위', '는', '15'...","4 위 는 16 , 5 위 는 15 , 1 쳤다"


In [13]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,correct_str,morphs,joined_morphs,joined_sentences
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.,"['블로그', 'mom', '5', '개', '의', '글', 'mom', '목록'...",블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 .,블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 . mom 2...
1,momo 2018 12 31 19 52 https blog naver com omj...,0,"mom 2018, 12, 31, 19, 52 https blog NAVER com ...","['mom', '2018', ',', '12', ',', '31', ',', '19...","mom 2018 , 12 , 31 , 19 , 52 https blog NAVER ...",블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 . mom 2...
2,An enemy generally says and believes what he w...,0,An enemy generally says and believes what he w...,"['An', 'enemy', 'generally', 'says', 'and', 'b...",An enemy generally says and believes what he w...,블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 . mom 2...
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0,True love is the joy of life 진실한 사랑은 인생의 환희다,"['True', 'love', 'is', 'the', 'joy', 'of', 'li...",True love is the joy of life 진실한 사랑 은 인생 의 환희 다,블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 . mom 2...
4,Carpe diem 현재를 즐겨라,0,Crape idem 현재를 즐겨라.,"['Crape', 'idem', '현재', '를', '즐겨라', '.']",Crape idem 현재 를 즐겨라 .,블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 . mom 2...
...,...,...,...,...,...,...
135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다,5170,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.,"['한편', '경계', '의', '의미', '로', '이', '사자성어', '를',...",한편 경계 의 의미 로 이 사자성어 를 추천 한 이 들 도 눈 에 띈다 .,"블로그 경제 사회 동향 2 , 516 개 의 글 경제 사회 동향 목록 열기 경제 사..."
135820,조은영 원광대 교수 미술과 는 2017년을 종합하기에는 수락석출 외의 단어들이 지나...,5170,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...,"['조은영', '원광대', '교수', '미술', '과는', '2017년', '을',...",조은영 원광대 교수 미술 과는 2017년 을 종합 하기에는 수락 석출 외 의 단어 ...,"블로그 경제 사회 동향 2 , 516 개 의 글 경제 사회 동향 목록 열기 경제 사..."
135821,올해의 사자성어는 3위부터 5위까지는 약 16 대의 고른 분포를 보인 것이 특징이다,5170,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.,"['올해', '의', '사자성어', '는', '3', '위', '부터', '5', ...",올해 의 사자성어 는 3 위 부터 5 위 까지는 약 16 대의 고른 분포 를 보인 ...,"블로그 경제 사회 동향 2 , 516 개 의 글 경제 사회 동향 목록 열기 경제 사..."
135822,4위는 16 5위는 15 1 였다,5170,"4위는 16, 5위는 15, 1 쳤다","['4', '위', '는', '16', ',', '5', '위', '는', '15'...","4 위 는 16 , 5 위 는 15 , 1 쳤다","블로그 경제 사회 동향 2 , 516 개 의 글 경제 사회 동향 목록 열기 경제 사..."


In [14]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,블로그 mom 5 개 의 글 mom 목록 열기 영어 명언 모음 입니다 . mom 2...
1,1,인사말 https blog NAVER com 221222451585 인사말 안녕하세...
2,2,블로그 게시판 52 개 의 글 게시판 목록 열기 콩 그 는 새우는가 마취 들 되어 ...
3,3,블로그 복지 단체 자선가 146 개 의 글 복지 단체 자선가 목록 닫기 복지 단체 ...
4,4,4 급 항만 경제 전략 국장 한 병수 도시 주택 국장 손정호 5 급 항만 정책 과장...
...,...,...
5166,5166,블로그 공지 목록 공지 글 글 제목 작성 일 1 공지 2025 학년 도 의대 의예 ...
5167,5167,"포토 로그 43 , 826 개 의 글 포토 로그 목록 열기 제주 환경 살리기 시민 ..."
5168,5168,블로그 공지 목록 공지 글 글 제목 작성 일 481 공지 독자 모집 한 주간 의 농...
5169,5169,숲 Issue 공지 목록 공지 글 글 제목 작성 일 공지 에코 힐링 워킹 프로그램 ...


In [15]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [16]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [17]:
tdm.shape

(5171, 134289)

In [18]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02103273, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08147005, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02135049, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [19]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)

In [20]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [21]:
tfidf_DF

,0,00,000,0001,0001076717,0001109380,0001287508,0002,000206,0002199342,...,힘차고,힘차며,힘차지요,힘찬,힙,힙니,힙스터,힙합,힝,ｍ
0,0.000000,0.000000,0.0,0.012561,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.021033,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.003698,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5166,0.000000,0.006565,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5167,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5168,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5169,0.081470,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_DF = pd.read_csv("../dataset/본문_TF-IDF/2018산림복지_본문_TF-IDF행렬.txt", encoding='UTF-8')
tfidf_DF

In [ ]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

In [49]:
# 이제 다 완성된 TF-IDF 행렬은 txt로 저장해줌.
tfidf_DF.to_csv("../dataset/본문_TF-IDF/2018산림복지_본문_TF-IDF행렬.txt", index=False, encoding='UTF-8')

In [50]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,51.032181
1,00,50.503403
2,000,19.454386
3,0001,0.049739
4,0001076717,0.034615
...,...,...
134284,힙니,0.062174
134285,힙스터,0.058074
134286,힙합,0.405733
134287,힝,0.086247


In [51]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/본문_TF-IDF/2018산림복지_본문_TF-IDF합.txt", index=False, encoding='UTF-8')